In [0]:
import os
import mlflow
import yaml

from mlflow.models.resources import DatabricksVectorSearchIndex, DatabricksServingEndpoint
from databricks import agents

In [0]:
mlflow.set_registry_uri("databricks-uc")

with open("names.yaml", "r") as file:
    names = yaml.safe_load(file)
with open("rag_chain_config.yaml", "r") as file:
    config = yaml.safe_load(file)


TABLE_NAME = names.get("table_name")
MODEL_NAME = names.get("model_name")
VECTOR_SEARCH_ENDPOINT_NAME = names.get("vector_search_endpoint_name")
VS_INDEX_FULLNAME = names.get("vs_index_fullname")

CHAIN_CONFIG = mlflow.models.ModelConfig(development_config='rag_chain_config.yaml')

## Testing, registering and deploying model

In [0]:
databricks_resources = CHAIN_CONFIG.get("databricks_resources")
llm_config = CHAIN_CONFIG.get("llm_config")
retriever_config = CHAIN_CONFIG.get("retriever_config")
client = mlflow.MlflowClient()

with mlflow.start_run(run_name="databricks_docs_chatbot") as run:
  logged_chain_info = mlflow.langchain.log_model(
      lc_model=os.path.join(os.getcwd(), 'create_chain'),  # Chain code file e.g., /path/to/the/chain.py 
      model_config="rag_chain_config.yaml",
      artifact_path="chain",
      input_example=CHAIN_CONFIG.get("input_example"),
      #example_no_conversion=True,
      resources=[
        DatabricksVectorSearchIndex(index_name=retriever_config.get("vector_search_index")),
        DatabricksServingEndpoint(endpoint_name=databricks_resources.get("llm_endpoint_name"))
      ],
    extra_pip_requirements=["databricks-connect"]
  )

  client.log_artifact(run.info.run_id, "names.yaml", artifact_path="chain")
  client.log_artifact(run.info.run_id, "rag_chain_config.yaml", artifact_path="chain")

uc_registered_model_info = mlflow.register_model(model_uri=logged_chain_info.model_uri, name=MODEL_NAME)

In [0]:
deploys = agents.get_deployments(MODEL_NAME)
newest_version = max([int(version.version) for version in mlflow.MlflowClient().search_model_versions(f"name = '{MODEL_NAME}'")])

if deploys:
  print('Model already deployed. Checking versions...')
  current_deployed_version = int(deploys[-1].model_version)

  if current_deployed_version < newest_version:
    print(f'New version found. Deleting deployment from model version {current_deployed_version} and deploying version {newest_version}')
    agents.delete_deployment(MODEL_NAME, current_deployed_version)
    deployment_info = agents.deploy(MODEL_NAME, model_version=newest_version, scale_to_zero=False)
  else:
    print(f'Version {current_deployed_version} is most up-to-date and is already deployed.')

else:
  print('No deployment found. Creating Model Serving endpoint')
  deployment_info = agents.deploy(MODEL_NAME, model_version=newest_version, scale_to_zero=False)

In [0]:
# Add the user-facing instructions to the Review App
instructions_to_reviewer = f"""
  ## Instructions for Testing the Databricks Documentation Assistant chatbot
  Your inputs are invaluable for the development team. By providing detailed feedback and corrections, you help us fix issues and improve the overall quality of the application. We rely on your expertise to identify any gaps or areas needing enhancement.
"""

agents.set_review_instructions(MODEL_NAME, instructions_to_reviewer)
user_list = ["pedro.perdomo@databricks.com", "leonardo.farhat@databricks.com"]
agents.set_permissions(model_name=MODEL_NAME, users=user_list, permission_level=agents.PermissionLevel.CAN_QUERY)
deployment = agents.get_deployments(MODEL_NAME)[-1]
print(f'Review app url: {deployment.review_app_url}')
print(f'Query endpoint: {deployment.query_endpoint}')


In [0]:
# ORQUESTRACAO
# - QUANDO ATUALIZAR? AGENDAR SEMANALMENTE
# - QUANDO DESLIGAR/LIGAR? AGENDAR SCRIPT DIÁRIO

# EXTRA
# - APAGAR VERSOES ANTERIORES MENSALMENTE
# - MANDAR TUDO PARA DABS